<div style="border:solid green 2px; padding: 20px">
<b>Александр, привет!</b>

Меня зовут Евгений Дащенко, и я буду проверять твой проект. Предлагаю общаться на «ты» 😊 Но если это не удобно - дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать аналитиком данных. Ты уже проделал большую работу над проектом, но давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - пожалуйста, не перемещай, не изменяй и не удаляй их. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b> Так выделены самые важные замечания. Без их отработки проект не будет принят. </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера:</b> Так выделены небольшие замечания. Одна-два таких замечания в проекте допустимы, но если их будет больше - тебе будет необходимо внести исправления. Это как тестовое задание при приеме на работу: очень много мелких ошибок могут стать причиной отказа кандидату. 

</div>

<div class="alert alert-success">
<b>Комментарий ревьюера:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: если ты что-то меняешь в проекте или отвечаешь на мои комменатри — пиши об этом. Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Например, вот так.</div>

Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Здорово, когда есть вступление в сам проект, каждый сможет быстрее разобраться с тем о чем сам проект. Можно также привести краткий план проведения самой работы и описание используемых столбцов. 

</div>

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
data = pd.read_csv('/datasets/data.csv')
print(data.info())
print(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1  

<div class="alert alert-warning">
<b>Комментарий ревьюера:</b>

Рекомендую не применять метод `print()` для вывода таблиц. Лучше импортировать в
начале проекта метод `display` из библиотеки `IPython.display` и
применять его:

---
    
`from IPython.display import display`

`display(df)`

    
</div>

### Вывод

- children - 47 человек указали по -(минус)1 ребенку, исправим на 1, еще 76 чел. указали по 20 детей, что вызывает сомнения
- days_employed - отрицательные значения в колонке, пропущенные значения заменим на 0, т к эта колонка не участвует в анализе
- dob_years - 101 чел. указали, что им по 0 лет
- education - разный регистр, приведем к нижнему регистру для поиска дубликатов 
- family_status - разный регистр, приведем к нижнему регистру для поиска дубликатов 
- gender - 1 чел. указал XNA
- total_income - пропущены значения, заменим на значение медианы

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
data.isnull().sum()

# отрицательное кол-во детей заменили на положительное число
data['children'] = data['children'].replace(-1, 1)

# пропущенные значения заменим на 0 в таблице 'days_employed'
data['days_employed'] = data['days_employed'].fillna(0)

# переведем все в нижний регистр в таблице 'education'
data['education'] = data['education'].str.lower()

# переведем все в нижний регистр в таблице 'family_status'
data['family_status'] = data['family_status'].str.lower()

# рассчитаем значение медианы и заменим пропущениые значения этим числом
median_total_income = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(median_total_income)

# проверим на кол-во пропущенных значений
data.isnull().sum()



children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

А если бы мы посмотрели на долю значений детей с количеством -1, то смогли бы принять решение в сторону избавления от этих данных ? 
    
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В том числе метод `astype` и `fillna` можно применять к нескольким столбцам одновременно.
    
Если столбцов более 2–ух, то рекомендую применять цикл `for`.
    
</div>

In [12]:
# пример

data.fillna(
    {
        "total_income": data['total_income'].median(),
        "days_employed": 0,
    }
)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purpose,salary
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,очень высокий
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,высокий
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,очень высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,высокий
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость,очень высокий
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль,высокий
21451,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость,низкий
21452,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль,очень высокий


Методом isnull() определяем пропущенные значения, а потом для days_employed - значения заменим на 0, тк эта колонка не участвует в анализе,а для total_income - заменим на значение медианы. Причиной пропусков может быть сокрытие информации или потеря части данных.

### Замена типа данных

In [3]:
# заменим вещественные числа в total_income и days_employed на целые числа
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')

# проверка
data.info()
print(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1          -8437         42    высшее             0   
1         1          -4024         36   среднее             1   
2         0          -5623         33   среднее             1   
3         3          -4124         32   среднее             1   
4         0        

### Вывод

используем метод astype(), т к он позволяет перевести данные в необходиммый для нас тип

п.с. возможно необходимо перевести стаж в годы и избавится от отрицательных значений, для простоты восприятия, но эта колонка не участвует в анализе поэтому, наверное, в этом нет необходимости

### Обработка дубликатов

In [4]:
# поиск кол-ва дубликатов
data.duplicated().sum()

# удаление дубликатов
data = data.drop_duplicates().reset_index(drop=True)

# проверка
data.duplicated().sum()



0

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Хорошо, что удалили дубликаты, они нам ни к чему.
    
</div>

### Вывод

Использовался метод duplicated для поиска дубликатов, а для удаления drop_duplicates(). В представленной таблицы нет персональных данных(ФИО, например) по которым можно было бы найти дубликаты при ручном поиске. Возможно, дубликаты появились при обращении в разные отделения банка для офрмления кредита.

### Лемматизация

In [5]:
m = Mystem()
# находим уникальные значения в столбце 
lem_purpose = data['purpose'].unique()

# проверка и определение значений, по которым в дальнейшем будем категоризировать данные
print(lem_purpose)

# автомобиль, недвижимость/жилье, свадьба, образование

# в функции пропускаем весь столбец через лемматизацию и через цикл присваиваем новые значения в новом столбце
def lemmas_purpose(purpose):
    lemma = m.lemmatize(purpose)
    for value in lemma:
        if 'автомоб' in value:
            return 'автомобиль'
        elif 'образов' in value:
            return 'образование'
        elif 'свадьб' in value:
            return 'свадьба'
        elif 'жил' in value or 'недвиж' in value:
            return 'недвижимость'

    
data['lemmas_purpose'] = data['purpose'].apply(lemmas_purpose)

# проверка
data['lemmas_purpose'].value_counts()

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: lemmas_purpose, dtype: int64

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Категории кредитов опредилили верно 👍
    
</div>

### Вывод

сначала нашли уникальные значения в солбце 'purpose', распечатали и определелили, что основными категориями могут быть: автомобиль, недвижимость/жилье, свадьба и образование. Далее в функции пропускаем весь столбец через лемматизацию и через цикл находим соответствующие значения и присваиваем новые значения в новом столбце.


### Категоризация данных

In [6]:
# категоризация по уровню дохода
def salary(payday):
    if payday <= 100000:
        return 'низкий'
    elif 100000 < payday <= 140000:
        return 'средний'
    elif 140000 < payday <= 180000:
        return 'высокий'
    elif  payday > 180000:
        return 'очень высокий'
    
        
data['salary'] = data['total_income'].apply(salary)

#проверка
print(data['salary'].value_counts())


очень высокий    6422
высокий          5927
средний          4642
низкий           4463
Name: salary, dtype: int64


### Вывод

создали категоризацию по уровню дохода, а также выше была создана по целям кредита.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В том числе, чтобы определить категории по нашим доходам, мы можем обратиться к стандартным квантилям.
    
</div>

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
# кол-во семей с детьми
parent_child = data[data['children'] >= 1]['children'].count()

# кол-во семей без детей
parent_not_child = data[data['children'] == 0]['children'].count()

# кол-во семей с детьми, которые не имеют задолжности
parent_child_not_debt = data[(data['debt'] == 0) & (data['children'] >= 1)]['children'].count()

# кол-во семей без детей, которые не имеют задолжности
parent_not_child_not_debt = data[(data['debt'] == 0) & (data['children'] == 0)]['children'].count()

#найдем процентное соотношение 
interest_child = parent_child_not_debt / parent_child
interest_not_child = parent_not_child_not_debt / parent_not_child

print('Процент семей с детьми, которые не имеют задолжностей по кредиту {:.1%}'.format(interest_child))
print('Процент семей без детей, которые не имеют задолжностей по кредиту {:.1%}'.format(interest_not_child))
print()

#методом pivot_table создадим сводную таблицу для наглядного представления результатов
pivot_child = data.pivot_table(index=['children'], columns='debt', values='education', aggfunc='count')
# обработаем полученные результаты
pivot_child[0] = pivot_child[0].astype('int')
pivot_child[1] = pivot_child[1].fillna(0)
pivot_child[1] = pivot_child[1].astype('int')
# посчитаем долю людей с разным кол-вом детей, которые не имеют задолжности
pivot_child['interest_child'] = pivot_child[0] / (pivot_child[0] + pivot_child[1])
# представим в процентах
pivot_child['percent'] = pivot_child['interest_child'] * 100
# поменяем название столбцов
pivot_child.set_axis(['нет задолжн.','есть задолжн.','доля', 'проценты'], axis = 'columns', inplace = True) 

print(pivot_child)


Процент семей с детьми, которые не имеют задолжностей по кредиту 90.8%
Процент семей без детей, которые не имеют задолжностей по кредиту 92.5%

          нет задолжн.  есть задолжн.      доля    проценты
children                                                   
0                13028           1063  0.924562   92.456178
1                 4410            445  0.908342   90.834192
2                 1858            194  0.905458   90.545809
3                  303             27  0.918182   91.818182
4                   37              4  0.902439   90.243902
5                    9              0  1.000000  100.000000
20                  68              8  0.894737   89.473684


### Вывод

В семьях без детей процент вовремя выплаченных кредитов больше на 1,7%, чем в семьях имеющих детей. Возможно, это связано с достаточно дорогим содержанием детей, что приводит к чуть большему кол-ву задолжностей по кредиту, чем в семьях без детей.
В целом в семьях с разным кол-вом детей процент вовремя выплаченных кредитов ниже, чем в семьях без детей(исключение семьи, где 5 детей).

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Верно 👍
    
---
    
Лучше всего менять названия столбцов с помощью метода `rename` 
    
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
#методом pivot_table создадим сводную таблицу для наглядного представления результатов
pivot_family_status = data.pivot_table(index=['family_status'], columns='debt', values = 'gender', aggfunc='count')

# посчитаем долю людей с разным семейным положением, которые не имеют задолжности
pivot_family_status['family_stat'] = pivot_family_status[0] / (pivot_family_status[0] + pivot_family_status[1])

# представим в процентах
pivot_family_status['percent'] = pivot_family_status['family_stat'] * 100

# поменяем название столбцов
pivot_family_status.set_axis(['нет задолжн.','есть задолжн.','доля', 'проценты'], axis = 'columns', inplace = True) 



print(pivot_family_status)


                       нет задолжн.  есть задолжн.      доля   проценты
family_status                                                          
в разводе                      1110             85  0.928870  92.887029
вдовец / вдова                  896             63  0.934307  93.430657
гражданский брак               3763            388  0.906529  90.652855
женат / замужем               11408            931  0.924548  92.454818
не женат / не замужем          2536            274  0.902491  90.249110


<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В точку 👍
    
Обрати внимание, что наши действия повторяются с точки зрения расчетов.
    
Это говорит о том, что мы можем создать функцию для постоянных расчетов. Тем самым мы сможем оптимизировать свой код 😊
    
</div>

### Вывод

In [9]:
# люди побывавшие в браке(не гражданском) или находящиеся сейчас в браке, имеют задолжности реже на 2-3%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
#методом pivot_table создадим сводную таблицу для наглядного представления результатов
pivot_salary = data.pivot_table(index=['salary'], columns='debt', values = 'gender', aggfunc='count')

# посчитаем долю людей с разными доходами, которые не имеют задолжности
pivot_salary['salary level'] = pivot_salary[0] / (pivot_salary[0] + pivot_salary[1])

# представим в процентах
pivot_salary['percent'] = pivot_salary['salary level'] * 100

# поменяем название столбцов
pivot_salary.set_axis(['нет задолжн.','есть задолжн.','доля', 'проценты'], axis = 'columns', inplace = True) 



print(pivot_salary)

               нет задолжн.  есть задолжн.      доля   проценты
salary                                                         
высокий                5410            517  0.912772  91.277206
низкий                 4109            354  0.920681  92.068116
очень высокий          5944            478  0.925568  92.556836
средний                4250            392  0.915554  91.555364


### Вывод

можно сказать, что уровень дохода, практически не влияет на задолжность по кредиту, все результаты с разностью чуть более 1%.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Верно 👍
    
В том числе можно было посчитать с метода группировки `groupby` (будем иметь в виду как альтернативный метод)
    
</div>

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
#методом pivot_table создадим сводную таблицу для наглядного представления результатов
pivot_lemmas_purpose = data.pivot_table(index=['lemmas_purpose'], columns='debt', values = 'gender', aggfunc='count')

# посчитаем долю людей с разными целями кредита, которые не имеют задолжности
pivot_lemmas_purpose['purpos'] = pivot_lemmas_purpose[0] / (pivot_lemmas_purpose[0] + pivot_lemmas_purpose[1])

# представим в процентах
pivot_lemmas_purpose['percent'] = pivot_lemmas_purpose['purpos'] * 100

# поменяем название столбцов
pivot_lemmas_purpose.set_axis(['нет задолжн.','есть задолжн.','доля', 'проценты'], axis = 'columns', inplace = True) 



print(pivot_lemmas_purpose)

                нет задолжн.  есть задолжн.      доля   проценты
lemmas_purpose                                                  
автомобиль              3903            403  0.906410  90.640966
недвижимость           10029            782  0.927666  92.766627
образование             3643            370  0.907800  90.779965
свадьба                 2138            186  0.919966  91.996558


### Вывод

меньший процент c задолжностями по кредитам для недвижимости и свадьбы можно объяснить тем, что чаще всего их берут люди(семья), в которой несколько источников доходов (работающие муж и жена, например). А автомобили и образование чаще берут люди с одним источником доходов. 

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В точку 👍
    
В том числе можно было бы построить графики, например с помощью метода `hist` 
    
Как раз способы визуализации, мы уже будем проходить в следующем проекте 😊
    
</div>

### Шаг 4. Общий вывод

- В семьях без детей процент вовремя выплаченных кредитов больше на 1,7%, чем в семьях имеющих детей
- люди побывавшие в браке(не гражданском) или находящиеся сейчас в браке, имеют задолжности реже на 2-3%
- уровень дохода, практически не влияет на задолжность по кредиту, все результаты с разностью меньше 1%
- процент c задолжностями по кредитам для недвижимости и свадьбы менеьше на 1-2%, чем на автомобили и образование

Процент отданных вовремя кредитов находится в диапозоне от 90% до 93,4%(не берем 100%, где 5 детей в семье - маленькая выборка и уберем 89,4% где 20 детей, есть основания думать, что это ошибочные данные). Наиболее часто отдающие кредит люди - это люди без детей, бывшие в законном браке или состоящие на данный момент в браке и приобретающие недвежимость. При этом уровень дохода почти не влияет на задолжность.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Выводы понятны и логичны, а самое главное подкреплены выявленными фактами.
       
В целом сам проект выполнен на хорошем уровне! На протяжении всей работы чувствуется глубина проработки задачи и это безусловный плюс для нас. Было использовано большое количество методов, которые будут помогать тебе в дальнейших проектах.
        
Поздравляю с успешным завершением проекта 😊👍
    
И желаю успехов в новых работах 😊
    
    
</div>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.